# Getting starting with the GRIDmeter™ library

This jupyter notebook is an interactive tutorial. It walks through loading data, running the stratified sampling model, and plotting results. You'll run all the code yourself. Cells can be executed with `<shift><enter>`. If you feel so inspired, make edits to the code in these cells and dig deeper.


    

# Background -- why this library

The `gridmeter` library originated from a project lead by Recurve Analytics, Inc. and funded by the US Dept. of Energy designed to identify practical methods for analyzing energy efficiency which are robust to external shocks (e.g. COVID-19).  From June through September 2020, Recurve hosted a series of meetings with industry stakeholders to discuss methods for constructing comparison groups, with the goal of presenting a recommended standard method to the GRID working group, a subsidiary of the Linux Foundation for Energy.  During this time, Recurve developed Python software to implement stratified sampling and demonstrate its effectiveness in practice.  The `gridmeter` library is the open-source Apache-licensed output of this process and is available to be used by anyone. 

This notebook will take you through several things that this library can do.


## Note on tutorial scope and related libraries

This tutorial assumes the reader has properly installed python and has a basic working knowledge of python syntax and usage. 

The `gridmeter` library uses `pandas` data frames as its principle way of representing data, therefore all inputs and outputs are data frames; see [this tutorial](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html) for information on how to use `pandas`.  The `plotnine` library is used to make visualizations which can be embedded in a Jupyter notebook or saved as .pngs; see [this website](https://plotnine.readthedocs.io/en/stable/) for information on `plotnine`.

In [1]:
import subprocess
import sys

for package in ['gridmeter', 'pandas', 'plotnine']:
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotnine as pn
import gridmeter as gm

## Viewing documentation

You can view code documentation by appending a question mark to a function and then executing that code, as in the following example:

In [2]:
gm.Stratified_Sampling?

Init signature: gm.Stratified_Sampling(settings: 'Settings | None' = None)
Docstring:      <no docstring>
File:           /app/applied_data_science/gridmeter/gridmeter/gridmeter/_stratified_sampling/create_comparison_groups.py
Type:           type
Subclasses:     

# Problem definition

Typically one begins with two populations of meters: a **treatment group**, and a **comparison pool**.  The treatment group consists of those meters for which savings needs to be tracked, e.g. if they represent customers participating in an energy efficiency program.  The comparison pool consists of all of the available meters that are not part of the treatment group.  The goal is to select a **comparison group** which is a subset of the comparison pool, such that the control group is **similar** to the treatment group.  The comparison group will then provide a realistic counterfactual compared to the treatment group and can be used to accurately estimate the impact of a program.

# Setting up tutorial data

Ordinarily you would start with your own time-series meter data, but for our purposes we'll generate some very basic dummy data.  `gridmeter` provides some functions which generate populations of dummy meters with randomly varying base and peak usage levels.  We'll use the following code to generate a population of 1000 treatment meters and 10000 comparison pool meters, and specify some consumption values for the treatment group that differ from the default, so that the groups are differently distributed.  The treatment group is specified by filtering the comparison pool along one of the available features (currently `winter_usage`, `summer_uage`, and `annual_usage`).  For this example, we'll assume our treatment group targets customers with usage above the 50th percentile in both summer and winter.

Once the population has been generated, we can create feature data and longitudinal data.  These functions take a few minutes to run upon first execution, but the results are cached to disk so subsequent runs will be fast.

Note: the structure of the dataframes is extremely important going forward.

In [3]:
# Loadshape and feature data

n_treatment = 1000
n_pool = 10000

filter = lambda df: df[(df.summer_usage > df.summer_usage.quantile(0.5)) & (df.winter_usage > df.winter_usage.quantile(0.5))]
population = gm.DummyTreatmentPoolPopulation(n_treatment=n_treatment, n_pool=n_pool, 
                                              treatment_filter_function=filter)

df_features= population.features()
df_features_pool = df_features[df_features['set']=='pool']
df_features_treatment = df_features[df_features['set']=='treatment']
df_loadshape = population.features_monthly()

df_features_pool = df_features_pool.rename(columns={'meter_id':'id'}).drop(columns=['set'])
df_ls_pool = df_loadshape.loc[df_features_pool["id"]]
df_ls_pool = df_ls_pool.stack().reset_index().rename(columns={'meter_id':'id', 'month':'time', 0:'loadshape'})

df_features_treatment = df_features_treatment.rename(columns={'meter_id':'id'}).drop(columns=['set'])
df_ls_treatment = df_loadshape.loc[df_features_treatment["id"]]
df_ls_treatment = df_ls_treatment.stack().reset_index().rename(columns={'meter_id':'id', 'month':'time', 0:'loadshape'})

# Fake time series data
# Create a testing dataframe having an id, datetime of 1 month intervals, observed and modeled values 
num_intervals = 12  # 1 per month

# Create a DataFrame with 'id', 'datetime', 'observed', and 'modeled' columns
df_ts_treatment = pd.DataFrame({
    'id': np.repeat(df_features_treatment["id"].values, num_intervals),  # only 3 ids for easier comparison
    'datetime': pd.date_range(start='2023-01-01', periods=num_intervals, freq='M').tolist() * n_treatment, 
    'observed': np.random.rand(num_intervals * n_treatment),  # randomized
    'modeled': np.random.rand(num_intervals * n_treatment)  # randomized
})

INFO:root:Caching objects to .cache


In [4]:
# Loadshape dataframe format, columns: [id, time, loadshape]
# time depends on whatever you have aggregated to

print("Loadshape dataframe format:")
df_ls_treatment.head()

Loadshape dataframe format:


,id,time,loadshape
0,meter_8,1,1416.625014
1,meter_8,2,1338.381990
2,meter_8,3,1277.526603
3,meter_8,4,1217.073287
4,meter_8,5,1356.374682


In [5]:
# Time Series dataframe format, columns: [id, datetime, Optional[observed], Optional[modeled], Optional[error]]
# observed, modeled, and error are optional, but at least one must be present
# if you are using observed and modeled to calculate error, then you must have both observed and modeled

print("Time series dataframe format:")
df_ts_treatment.head()

Time series dataframe format:


,id,datetime,observed,modeled
0,meter_8,2023-01-31,0.201009,0.949077
1,meter_8,2023-02-28,0.267766,0.264981
2,meter_8,2023-03-31,0.342109,0.561197
3,meter_8,2023-04-30,0.456873,0.972374
4,meter_8,2023-05-31,0.987438,0.958791


In [6]:
# Features dataframe format, columns: [id, feature1, feature2, ...]
# features can be named whatever you like and you may include as many as you like

print("Features dataframe format:")
df_features_treatment.head()

Features dataframe format:


,id,winter_usage,summer_usage,annual_usage
8,meter_8,4474.540107,4498.474411,16999.617199
33,meter_33,4898.471826,5078.343347,18424.288094
56,meter_56,4629.797900,3975.296379,17700.280488
60,meter_60,4526.865188,3980.372332,17211.359791
62,meter_62,5053.708165,3974.311687,18117.419607


# Setting up data

Gridmeter uses the `Data` class as the inputs to all of the comparison group methods. Its purpose is to validate your data and to perform the necessary transformations to go into the comparison group methods. `Data` is instantiated with `Data_Settings`. Once instantiated the method `.set_data` can be called using one of `loadshape_df` and `time_series_df` as well as `features_df` depending upon your needs. If you input `time_series_df` it will compute the loadshape.

Method requirements:
- Clustering: Loadshape
- Individual Meter Matching: Loadshape
- Stratified Sampling: Loadshape/Features (depends upon settings)

In [7]:
data_settings = gm.Data_Settings(time_period='month')

In [8]:
# Let's make an invalid entry to see what happens
df_ts_treatment.loc[0, 'observed'] = np.nan

ts_data = gm.Data().set_data(time_series_df=df_ts_treatment)

ts_data.get_excluded_ids()

,id,reason
0,meter_10015,missing minimum number of values in loadshape_df
1,meter_10026,missing minimum number of values in loadshape_df
2,meter_10029,missing minimum number of values in loadshape_df
3,meter_10034,missing minimum number of values in loadshape_df
4,meter_10044,missing minimum number of values in loadshape_df
...,...,...
995,meter_9966,missing minimum number of values in loadshape_df
996,meter_997,missing minimum number of values in loadshape_df
997,meter_9974,missing minimum number of values in loadshape_df
998,meter_9992,missing minimum number of values in loadshape_df


In [9]:
# Set data classes
# for the purposes of this tutorial, we'll set both loadshapes and features

data_settings = gm.Data_Settings(AGG_TYPE=None, LOADSHAPE_TYPE=None, TIME_PERIOD=None)

pool_data = gm.Data(data_settings)
pool_data.set_data(loadshape_df=df_ls_pool, features_df=df_features_pool)

treatment_data = gm.Data(data_settings)
treatment_data.set_data(loadshape_df=df_ls_treatment, features_df=df_features_treatment)

In [10]:
treatment_data.get_loadshape()

,1,2,3,4,5,6,7,8,9,10,11,12
id,,,,,,,,,,,,
meter_10015,1487.075136,1772.160553,1700.007679,1373.618313,1399.705246,1208.550412,1583.023200,1272.234309,1735.385947,1390.719550,1714.475611,1420.198345
meter_10026,2027.067674,1727.620374,1494.155440,1385.250634,1335.169614,1282.868411,1194.246322,1375.576861,1570.446604,1230.476249,1567.337511,1602.583098
meter_10029,1240.466319,1482.362033,1382.311526,1402.749948,1540.491038,1336.607327,1242.334334,1420.454655,1559.761040,1560.235744,1443.091529,1476.363147
meter_10034,1373.233778,1560.863076,1174.082873,1595.585703,1356.832501,1415.234478,1348.946313,1375.653387,1283.188967,1499.160262,1540.950193,1334.519215
meter_10044,1181.522230,1318.197869,1663.735589,1581.444054,1830.537569,1175.091826,1297.401486,1202.654418,1660.670675,1590.550124,1801.845668,1280.122172
...,...,...,...,...,...,...,...,...,...,...,...,...
meter_9966,1713.278635,1583.993143,1470.902751,1643.762071,1685.749375,1531.277621,1387.402745,1144.465395,1525.362148,1340.070359,1441.718648,1455.741363
meter_997,1674.525334,1400.708446,1305.325420,1429.551442,1303.968237,1686.636852,1363.709231,1481.356785,1321.017955,1049.142803,1206.350984,1316.598729
meter_9974,1623.680467,1392.346245,1525.522440,1157.463018,1155.465955,1456.242137,1469.635275,1087.861268,1239.333549,1541.160557,1467.633079,1279.468672


In [11]:
treatment_data.get_features()

,winter_usage,summer_usage,annual_usage
id,,,
meter_8,4474.540107,4498.474411,16999.617199
meter_33,4898.471826,5078.343347,18424.288094
meter_56,4629.797900,3975.296379,17700.280488
meter_60,4526.865188,3980.372332,17211.359791
meter_62,5053.708165,3974.311687,18117.419607
...,...,...,...
meter_10957,4439.195070,4510.355083,17054.444500
meter_10966,4428.247078,4508.444470,16918.332693
meter_10985,4746.081470,4247.471266,17088.210893


In [12]:
data_settings

Data_Settings(AGG_TYPE=None, LOADSHAPE_TYPE=None, TIME_PERIOD=None, INTERPOLATE_MISSING=True, MIN_DATA_PCT_REQUIRED=0.8, SEASON=Season_Definition(JANUARY='winter', FEBRUARY='winter', MARCH='shoulder', APRIL='shoulder', MAY='shoulder', JUNE='summer', JULY='summer', AUGUST='summer', SEPTEMBER='summer', OCTOBER='shoulder', NOVEMBER='winter', DECEMBER='winter', OPTIONS=['summer', 'shoulder', 'winter']), WEEKDAY_WEEKEND=Weekday_Weekend_Definition(MONDAY='weekday', TUESDAY='weekday', WEDNESDAY='weekday', THURSDAY='weekday', FRIDAY='weekday', SATURDAY='weekend', SUNDAY='weekend', OPTIONS=['weekday', 'weekend']))

# Clustering

Is it fast? Not particularly, but it has advantages

In [13]:
clustering_settings = gm.Clustering_Settings(TREATMENT_MATCH_LOSS="MAE", USE_MULTIPROCESSING=True)
clustering = gm.Clustering(clustering_settings)
df_cg, df_t_coeffs = clustering.get_comparison_group(treatment_data, pool_data)
df_cg

,cluster
id,
meter_6586,-1
meter_6575,-1
meter_6576,-1
meter_6579,-1
meter_658,-1
...,...
meter_3933,47
meter_9619,47
meter_7153,47


In [ ]:
df_t_coeffs

In [ ]:
df_cg

# Individual Meter Matching
For Euclidean distance matching, the usage patterns of each treatment meter are compared to the usage patterns of each comparison pool meter in order to find the closest matches. The subsequent group of meters that have been "matched" become the resulting comparison group.

## Features For Matching

Any usage pattern can be used for euclidean distance matching. If an 8760 hourly trace is available, it is suggested to use a 'seasonal-hour-of-week' load-shape (3 168-point load shapes, one for summer, winter, and shoulder). If only monthly data is available, it is suggested to use the 12 months of usage as the usage pattern.


In [ ]:
# Test IMM

imm_settings = gm.IMM_Settings()
imm = gm.IMM(imm_settings)
df_cg, df_t_coeffs = imm.get_comparison_group(treatment_data, pool_data)

In [ ]:
imm.plot_comparison_group_loadshape()

# Stratified Sampling Matching

There are many ways to define similarity and many methods for constructing similar groups.  The stratified sampling approach was selected in particular because it offers a good balance in terms effectiveness, ease of implementation, and simplicity. We define stratified sampling as follows:

- Each meter is represented by a vector of numerical features, e.g. annual usage, summer peak usage, etc
- A small number of features, typically between 1 and 3 inclusive, are selected: this is the number of *dimensions*.
- The treatment group is divided up into a set of multidimensional bins according to a predefined binning configuration.
- The proportion of treatment meters present in each bin is computed.
- A set of meters is sampled from the comparison pool such that the sample is distributed amongst the bins in the same proportion as the treatment group.  The sample is called the comparison group.
- The treatment and comparison groups are compared using additional meter data, e.g. load shapes.
- (Optional) An optimal binning configuration is selected which minimizes error between treatment and control groups.


In order to use this library, it is therefore necessary to have the following data at hand:
- A data frame containing the treatment group, with one ID column, and a set of numerical feature columns, one row per meter;
- A data frame containing the comparison pool, in the same format;
- A data frame containing the treatment group data for equivalence, with one ID column, one feature name column, and one value column; e.g. for an annual monthly load shape, the feature name column might contain the month name, and there might be twelve rows per meter;
- A data frame containing the comparison pool data for equivalence, in the same format.

## Selecting features for stratification

Normally, you will compute a set of features from your time series meter traces, and then select a subset of those features to stratify on.  The feature computation step is up to you.  We recommend running the Caltrack model to derive heating and cooling coefficients and deriving features from these outputs.  Other techniques such as principle component analysis could be useful as well.  In general, aim for stratifying on between one and three features, and aim for orthogonal features if possible.  In the dummy data, there are three features available: `summer_usage`, `winter_usage`, and `annual_usage` which are shown in the scatter plots below.  `annual_usage` is correlated with the other two, as you would expect, so we will stratify based on `summer_usage` and `winter_usage`.



In [ ]:
pd.plotting.scatter_matrix(df_features_treatment);

By plotting the two features below we can see that the treatment group and comparison pool are differently distributed, such that a random sample from the pool would not be a good counterfactual for the treatment group -- this is why we need stratified sampling.

In [ ]:
pn.ggplot(df_features, pn.aes(x='summer_usage', y='winter_usage', color='set')) + pn.geom_point(alpha=0.5) + pn.facet_wrap("~set") + pn.theme_bw()

To use stratified sampling, you must supply input data, a list of input columns to stratify on, and some configuration, according to the example below.   Additionally, you have the option of using the library to select an optimal configuration for you.  We'll first demonstrate the basic usage of the model, in which you supply all of the configuration information.

### Number of outputs 

The size of the comparison group is constrained by the size of the pool and by the degree to which the treatment group overlaps the pool in terms of the stratification columns.  For example, if the treatment group and pool are disjoint, no comparison group can be constructed; if only 5% of the pool overlaps the treatment group, then the effective available pool is constructed from that 5% of overlapping meters.  Generally, if you do not specify the output size, `gridmeter` will construct the largest possible comparison group up to 5000 meters.   

In [ ]:
strat_col_settings = [
    {
        "column_name": "summer_usage", 
        "n_bins": 8, 
        "min_value_allowed": 3000, 
        "max_value_allowed": 6000, 
        "fixed_width": False
    },
    {
        "column_name": "winter_usage",
        "n_bins": 8, 
        "min_value_allowed": 3000, 
        "max_value_allowed": 6000, 
        "fixed_width": False
    },
]

ss_settings = gm.stratified_sampling_settings(stratification_column=strat_col_settings, random_seed=1)
ss = gm.Stratified_Sampling(ss_settings)

df_clusters, df_treatment_weights = ss.get_comparison_group(treatment_data, pool_data)

In [ ]:
# Original Stratified Sampling dataframe
ss.df_raw

In [ ]:
# bin count
ss.diagnostics().count_bins()

## Viewing diagnostic plots 

Once the model has been fit, you can view several plots which will show the treatment group, comparison pool, and the final sample (i.e. the comparison group).  If the model has worked correctly, then the sample should be distributed similarly to the treatment group.  There are three plots available:

1. Histograms, showing the distribution of each stratification parameter in one dimension;
2. Scatter plots, showing an XY-plot of data points for each pair of dimensions;
3. Quantile plots with equivalence, showing the quantile distribution of each dimension, along with t-test and ks-test p-values comparison the treatment and comparison groups.  

The t- and ks-tests can be used as a measure of equivalence, however we recommend using equivalence-based optimization discussed below for better results in practice.  

In [ ]:
d = ss.diagnostics()

In [ ]:
d.histogram()

In [ ]:
d.scatter()

In [ ]:
d.quantile_equivalence()

## Equivalence-based optimization

The above method of stratified sampling is designed to match the distributions of one or more stratification parameters.  However, often in practice the desired outcome is to have a comparison group with similar usage patterns to a treatment group as measured through a load shape or similar set of time-series features.  To ensure that the load shapes are equivalent, you can use the `StratifiedSamplingBinSelector` which selects an optimal binning arrangement which minimizes load shape error between the treatment and comparison groups. 

In the following example, a bin selector is used to match on monthly load shapes.  

In [ ]:
strat_col_settings = {
    "equivalence_method": "chisquare",
    "min_n_bins": 2,
    "max_n_bins": 8,
    "stratification_column": [{
            "column_name": "summer_usage", 
            "n_bins": None, 
            "min_value_allowed": 3000, 
            "max_value_allowed": 6000, 
            "fixed_width": False
        },
        {
            "column_name": "winter_usage",
            "n_bins": None, 
            "min_value_allowed": 3000, 
            "max_value_allowed": 6000, 
            "fixed_width": False
    }],
    "random_seed": 1,
}

dss_settings = gm.distance_stratified_sampling_settings(**strat_col_settings)
dss = gm.Stratified_Sampling(dss_settings)

df_clusters2, df_treatment_weights2 = dss.get_comparison_group(treatment_data, pool_data)

In [ ]:
dss.model_bin_selector.plot_records_based_equiv_average()

The graph above shows the monthly load shape of the treatment group (red), the selected comparison group (black), other candidate comparison groups (blue), and the comparison pool (dots).  In this case, all of the candidates were a good match, so the selection process was probably unnecessary, however in more complex cases this method can be useful.  The final binning arrangement is available in the model object as follows:

In [ ]:
dss.diagnostics().scatter()

In [ ]:
dss.df_raw

This concludes the tutorial.  For questions, comments, or issues, please raise a Github issue here:  
https://github.com/recurve-methods/comparison_groups/issues

We thank you for your interest and participation in this project and look forward to your feedback.